In [3]:
import numpy as np;
import pandas as pd;
from numpy import loadtxt
from numpy import savetxt

## Initialize Parameters

In [4]:
## Parameters and Global variables
district = "all"


## Load and Initialize Data

In [7]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_population_data.csv")

## Number of regions
r = len(initDataDF.index)

## T-Matrix
tMatrix = loadtxt("../output/" + district + "_tmatrix_results.csv",delimiter=',')

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
initDataDF.reset_index(drop=True, inplace=True)

initDataDF.tail(300).head(10)

,name,type,N
735,Perambra__Kozhikode,P,32784
736,Peravoor__Kannur,P,23558
737,Perayam__Kollam,P,21236
738,Perinadu__Kollam,P,33955
739,Peringammala__Thiruvananthapuram,P,27667
740,Peringara__Pathanamthitta,P,21001
741,Peringom-Vayakkara__Kannur,P,29374
742,Peringottukurissi__Palakkad,P,25075
743,Perinjanam__Thrissur,P,21012
744,Perinthalmanna(M)__Malappuram,M,49723


## Prepare T-Matrix

In [9]:
tColSum = tMatrix.sum(axis=0)
tColSum

tRowSum = tMatrix.sum(axis=1)
tRowSum

tColSum
#tMatrix[0]/initDataDF['N']
#tMatrix[1]/tColSum[0]
#np.multiply(tMatrix[0]/Ni,tMatrix[1]/tColSum).sum()

array([ 7116.24049744, 30962.63356938, 33086.67590358, ...,
       10060.05741336, 29005.93542084, 17897.37937073])

## Function Definition

In [10]:
def getCijv3(i, j, tMatrix, Ni):
    #print("[getCijv3]", )
    cwSum = np.multiply(tMatrix[i]/Ni,tMatrix[j]/tColSum).sum()
    return cwSum

## Compute C-Matrix

In [11]:
data = []
for i in range(0,r):
    Ni = initDataDF.iloc[i]['N']

    for j in range(0,r):
        #print("j=", j)
        Cij = getCijv3(i,j,tMatrix, Ni)
        #print("C(",i,",",j,") ", Cij)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Cij})

resultsDF = pd.DataFrame(data);


## Save Output

In [12]:
r = len(initDataDF.index)
C_TPart_Matrix = resultsDF['Cij'].to_numpy().reshape(r,r)
Wn = 15*C_TPart_Matrix
Wn
Wn.sum(axis=1)


array([15., 15., 15., ..., 15., 15., 15.])

In [13]:
resultsDF.to_csv("../output/" + district + "_cmatrix_results.csv")

In [11]:
resultsDF.head(10)

,iName,jName,Cij
0,ARNagar__Malappuram,ARNagar__Malappuram,0.854012
1,ARNagar__Malappuram,Adat__Thrissur,0.000053
2,ARNagar__Malappuram,Adichanalloor__Kollam,0.000003
3,ARNagar__Malappuram,Adimali__Idukki,0.000012
4,ARNagar__Malappuram,Adoor(M)__Pathanamthitta,0.000006
5,ARNagar__Malappuram,Agali__Palakkad,0.000046
6,ARNagar__Malappuram,Aiykkaranadu__Ernakulam,0.000007
7,ARNagar__Malappuram,Ajanur__Kasaragod,0.000014
8,ARNagar__Malappuram,Akalakunnam__Kottayam,0.000004
9,ARNagar__Malappuram,Akathethara__Palakkad,0.000038
